# M5 Forecasting: Final Notebook

# **Contents**

<h3> 1. Define helper functions </h3> 

<h3> 2. Create Final Pipelines </h3>

<h3> 3. Check the output </h3> 

In [21]:
#import dependencies
import pandas as pd
import numpy as np
import seaborn as sns
import random 
import datetime
from tqdm import tqdm
import pickle
import warnings
warnings.filterwarnings('ignore')

# 1. Define helper functions

## 1.1 Downcast Dataframe

In [2]:
#refer - https://stackoverflow.com/questions/1658714/how-to-get-the-range-of-valid-numpy-data-types
def downcast(data):
    cols = data.columns
    for col in cols:
        if data[col].dtype == object:
            if col =='date':
                data[col] = pd.to_datetime(data[col])
            else: 
                data[col] = data[col].astype('category') 
 
        #only check the upper value because we only have positive values in dataframes
        elif data[col].dtype == int: 
            if data[col].max() < np.iinfo('int8').max:
                data[col] = data[col].astype('int8')

            elif data[col].max() < np.iinfo('int16').max:
                data[col] = data[col].astype('int16')

            elif data[col].max() < np.iinfo('int32').max:
                data[col] = data[col].astype('int32')
            else:
                data[col] = data[col].astype('int64')

        elif data[col].dtype == float:
            if data[col].max() < np.finfo('float16').max:
                data[col] = data[col].astype('float16')
            elif data[col].max() < np.finfo('float32').max:
                data[col] = data[col].astype('float32')
            else:
                data[col] = data[col].astype('float64')

    return data

## 1.2 Compute Weights

In [3]:
level_groupings = {2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"],
                   8: ["store_id", "cat_id"], 9: ["store_id", "dept_id"], 10: ["item_id"], 11: ["item_id", "state_id"]}

def calculate_weightsL12(sales, cal, price, last28):
    ''' Calculate weights for level 12 (Product-Store) series' using the last 28 days sales data '''

    #calculating weights for level 12 : 'item_id, store_id' using last 28 days of train data
    #this loop is repeated 28 times to get the sales revenue of all ids for each of last 28 days 
    #flow of execution : day-> week id-> sell price of ids on the day-> sales revenue of ids for the day
    for day in range(last28[0], last28[1]):
        #get the week id corresponding to the day 
        week_id = int(cal[cal["d_"]==day]["wm_yr_wk"]) 

        #get the week price for each of the items corresponding to the week id   
        week_price = price[price["wm_yr_wk"]==week_id]

        #merge sales with week price on 'id'
        #note: we merge the dataframes using inner join so the id which are present in both dataframes will be retained after merging
        sales = sales.merge(week_price[["sell_price", "id"]], on=["id"], how='inner')

        #create a column which shows the sales revenue for the day
        #sales revenue = sell_price * units_sold 
        sales["sales_revenue_d_" + str(day)] = sales["sell_price"] * sales["d_" + str(day)]

        #drop the sell_price column
        sales.drop(columns=["sell_price"], inplace=True)

    #Sum of sales revenue of each id for last 28 days   1`
    sales_revenue_cols = [x for x in sales.columns if x.find("sales_revenue")==0]
    sales['sales_revenue_alldays'] = sales[sales_revenue_cols].sum(axis=1)

    #Compute weights for each Level 12 Time Series
    sales['weight'] = (1/12)*(sales['sales_revenue_alldays']/sales['sales_revenue_alldays'].sum())

    #Drop the unnecessary columns 
    sales.drop(columns = sales_revenue_cols+['sales_revenue_alldays'], inplace=True)

    return sales

def calculate_weightsALL(sales, levels):
    ''' Calculate weights for series' in rest of the aggregation levels '''
    
    #weights for level 1 : 'all'
    agg = pd.DataFrame(sales[[x for x in sales.columns if x.find("d_") == 0 or x.find("F") == 0]].sum()).transpose() 
    id_cols = ["item_id", "dept_id", "cat_id", "store_id", "state_id"]
    for col in id_cols:
        agg[col] = 'all'
    agg["level"] = 1
    agg["weight"] = 1/12
    column_order = agg.columns

    #weights for the rest of the levels (levels 2-11)
    for level in level_groupings:
        temp_df = sales.groupby(by=level_groupings[level]).sum().reset_index()
        temp_df["level"] = level
        
        for c in column_order:
            if c not in temp_df.columns:
                temp_df[c] = 'all'
                
        agg = agg.append(temp_df[column_order])

    return agg

## 1.3 Compute WRMSSE

In [4]:
def get_day_splits(sales, train_start, train_end, val_start, val_end):
    '''Create lists of days/columns to be selected for train, val and forecast when calculating RMSSE'''
    
    train_days =  [x for x in sales.columns if x.find("d_") == 0 and int(x.split("_")[1]) in range(train_start, train_end+1)] 
    val_days =  [x for x in sales.columns if x.find("d_") == 0 and int(x.split("_")[1]) in range(val_start, val_end+1)]
    forecast_days = [x for x in sales.columns if x.find("F") == 0]
    return train_days, val_days, forecast_days

def RMSSE(ground_truth, forecast, train, n, h):
    ''' Calculates the RMSSE score for all series in the dataframe. '''

    num = ((ground_truth - forecast)**2).sum(axis=1)
    den = 1/(n-1) * ((train[:, 1:] - train[:, :-1]) ** 2).sum(axis=1)  
    rmsse = (1/h * num/den) ** 0.5

    return rmsse

def WRMSSE(sales, agg, train_days, val_days, forecast_days, n, h):
    ''' Calculates the WRMSSE score for the model prediction '''
    
    ground_truth_df = np.array(sales[val_days])
    forecast_df = np.array(sales[forecast_days])
    train_df = np.array(sales[train_days])

    ground_truth_agg_df = np.array(agg[val_days])
    forecast_agg_df = np.array(agg[forecast_days])
    train_agg_df = np.array(agg[train_days])
            
    # calculate rmsse for each series using df[val-days] - df[forecast-days] ->numerator   
    sales["rmsse"] = RMSSE(ground_truth_df, forecast_df, train_df, n, h)
    agg["rmsse"] = RMSSE(ground_truth_agg_df, forecast_agg_df, train_agg_df, n, h)

    sales["wrmsse"] = sales["weight"] * sales["rmsse"]
    agg["wrmsse"] = agg["weight"] * agg["rmsse"]

    wrmsse = sales["wrmsse"].sum() + agg["wrmsse"].sum()
    
    return wrmsse

## 1.4 Feature Engineering
- We create 2 separate feature engineering functions for Validation and Evaluation days because the inputs and order of steps differ a bit.

## 1.4.1 Feature Engineering (Evaluation Days: 1942 to 1969)

In [56]:
def featEng_eval(X):
    ''' Takes list of level 12 ids as input and returns long format data with all features for days in evaluation phase '''
    
    train_end = 1941
    
    df_s = pd.read_csv('sales_train_evaluation.csv')
    df_p = pd.read_csv('sell_prices.csv')
    df_c = pd.read_csv('calendar.csv', parse_dates=['date'])
    
    '''
    Basic Data Manipulations: 
    1. Convert the day column names from string to number (Sales Dataframe)
    2. Add days/columns for evaluation period (Sales Dataframe)
    3. Convert day column dtype from object to integer and replace null values in event columns (Calender Dataframe)
    4. Create id feature combining store and item id (Price Dataframe) 
    '''
    df = df_s.loc[df_s['id'].isin(X)]
    
    #1.
    df.columns = list(df.columns[:6]) + list(range(1,train_end+1))

    #2.
    for day in range(train_end+1, train_end+28+1):
        df[day] = 0

    #3.
    df_c["d"]= df_c["d"].apply(lambda x: int(x.split("_")[1])).astype('int16')

    cal_catFeat = ['event_name_1','event_type_1','event_name_2','event_type_2']

    for feat in cal_catFeat:
        df_c[feat] = df_c[feat].replace(np.nan, -1)
    
    #4.
    df_p["id"] = df_p["item_id"] + "_" + df_p["store_id"] + "_evaluation"
    
    
    '''
    Basic Feature Creation (Time Based features):
    1. Derive quarter, week and day features from date.
    2. Create a binary is_weekend feature whose function is self explanatory    
    '''
    #1.
    datetimeFeat = ["quarter", "week", "day"] 
    for feat in datetimeFeat:
        df_c[feat] = getattr(df_c['date'].dt, feat).astype('int8')

    df_c.rename(columns = {'day':'day_of_month'}, inplace=True)

    #2.
    df_c['is_weekend'] = df_c["weekday"].apply(lambda x: 1 if x in ['Saturday','Sunday'] else 0).astype('int8')
    
    
    '''
    Transformations and Feature Creation:
    1. Convert the data to long format
    2. Create lag features derived from sales
    3. Create rolling window based statistical features on sales (mean)
    4. Merge dataframes
    5. Create price lag and change features 
    '''
    #1.
    df = pd.melt(df, id_vars = [c for c in df.columns if type(c)==str], var_name='d', value_name='sales')
    df['d'] = pd.to_numeric(df['d']).astype('int16')
    
    #drop majority of historical data to reduce computation time of features
    #because we only need preprocessed data for validation days we can drop most of previous data
    df = (df[df['d']>=1700]).reset_index(drop=True)
    
    #2.
    lags = [28, 30, 31, 35, 42, 49, 56, 63, 70, 77]
    
    for lag in tqdm(lags):
        df[f'lag_{lag}'] = df.groupby(["id"])["sales"].shift(lag).astype('float16')
        
    #3.
    windows = [7, 14, 28, 30, 45, 60, 90, 120]
    
    for window in tqdm(windows):
        df[f"rmean_28_{window}"] = df.groupby(["id"])["lag_28"].transform(lambda x: x.rolling(window).mean()).astype('float16')
    
    #4.
    df = pd.merge(df, df_c.drop(columns=['weekday']), how='left', on='d')
    df = pd.merge(df, df_p.drop(columns=['store_id', 'item_id']), how='left', on=['id', 'wm_yr_wk'])
    
    #5. 
    df['price_shift_t1'] = df.groupby(['id'])['sell_price'].shift(1)
    df['price_change_t1'] = (df['sell_price'] - df['price_shift_t1'])/df['price_shift_t1']
    
    
    '''
    Data Manipulation:
    1. Retain data after day 1941 and drop the rest
    2. Convert dtype of lag features to 'int16' 
    3. Replace null values in price based features with 0 
    '''
    #1.
    df = (df[(df['d']>=(train_end+1)) & (df['d']<=(train_end+28))]).reset_index(drop=True)
    
    #2.
    for feat in ["lag_28", "lag_30", "lag_31", "lag_35", "lag_42", "lag_49", "lag_56", "lag_63", "lag_70", "lag_77"]:
        df[feat] = df[feat].astype('int16')
    
    #3.
    for feat in ['sell_price','price_shift_t1','price_change_t1']:
        df[feat] = df[feat].replace(np.nan, 0)
    
    '''
    Categorical Feature Encoding:
    1. Load the category dicts 
    2. Use category dicts to encode categorical features
    '''
    sales_catFeat = ['id','item_id','dept_id','cat_id','store_id','state_id']
    cal_catFeat = ['event_name_1','event_type_1','event_name_2','event_type_2']

    #1.
    for feat in sales_catFeat+cal_catFeat:
        varStr = f'dict_{feat}'
        var = vars()
        var[varStr] = pickle.load(open(f"saved_dicts/dict_{feat}.pkl", "rb"))
    
    #2.
    for feat in sales_catFeat+cal_catFeat:
        varStr = f'dict_{feat}'
        df[feat] = df[feat].apply(lambda x: var[varStr][x] if type(x)==str else -1)
    
    '''
    Downcasting Dataframe
    '''
    df = downcast(df)
    
    return df

## 1.4.2 Feature Engineering (Validation Days: 1914 to 1941)

In [45]:
def featEng_val(X):
    ''' Takes level 12 series sales data in wide format as an input and returns long format data with added features for days 
        in validation phase '''

    train_end = 1913
    
    df_sales = X.copy()
    df_price = pd.read_csv('sell_prices.csv')
    df_cal = pd.read_csv('calendar.csv', parse_dates=['date'])

    '''
     Data Manipulations and Feature Creation: 
     1. Convert the day column names from string to number (Sales Dataframe)
     2. Add days/columns for evaluation period (Sales Dataframe)
     3. Convert day column dtype from object to integer and add time based features (Calender Dataframe)
     4. Create id feature combining store and item id (Price Dataframe) 
     '''
    #1.
    df_sales.columns = list(df_sales.columns[:6]) + list(range(1,train_end+1))
    
    #2.
    for day in range(train_end+1, train_end+28+1):
        df_sales[day] = 0
    
    #3.
    df_cal["d"]= df_cal["d"].apply(lambda x: int(x.split("_")[1])).astype('int16')
    
    datetimeFeat = ["quarter", "week", "day"] 
    for feat in datetimeFeat:
        df_cal[feat] = getattr(df_cal['date'].dt, feat).astype('int8')

    df_cal.rename(columns = {'day':'day_of_month'}, inplace=True)

    df_cal['is_weekend'] = df_cal["weekday"].apply(lambda x: 1 if x in ['Saturday','Sunday'] else 0).astype('int8')

    #4.
    df_price["id"] = df_price["item_id"] + "_" + df_price["store_id"] + "_evaluation"

    
    '''
    Downcasting Dataframes
    '''
    df_sales = downcast(df_sales)
    df_cal = downcast(df_cal)
    df_price = downcast(df_price)

    
    '''
    Feature encoding
    '''
    sales_catFeat = ['id','item_id','dept_id','cat_id','store_id','state_id']
    cal_catFeat = ['event_name_1','event_type_1','event_name_2','event_type_2']
    
    for col in sales_catFeat:
        df_sales[col] = df_sales[col].cat.codes
        if col == 'id':
            df_price[col] = df_price[col].cat.codes

    for col in cal_catFeat:
        df_cal[col] = df_cal[col].cat.codes

        
    '''
    Downcasting Dataframes
    '''
    df_sales = downcast(df_sales)
    df_cal = downcast(df_cal)
    df_price = downcast(df_price)

    
    '''
     Transformations and Feature Creation:
     1. Convert the data to long format
     2. Create lag features derived from sales
     3. Create rolling window based statistical features on sales (mean)
     4. Merge dataframes
     5. Create price lag and change features 
     '''
    #1.
    df_long = pd.melt(df_sales, id_vars = ['id','item_id','dept_id','cat_id','store_id','state_id'], var_name='d', value_name='sales')
    df_long['d'] = pd.to_numeric(df_long['d']).astype('int16')
    
    #drop majority of historical data to reduce computation time of features
    #because we only need preprocessed data for evaluation days we can drop most of previous data
    df_long = (df_long[df_long['d']>=1700]).reset_index(drop=True)
    
    #2.
    lags = [28, 30, 31, 35, 42, 49, 56, 63, 70, 77]
    for lag in tqdm(lags):
        df_long[f'lag_{lag}'] = df_long.groupby(["id"])['sales'].shift(lag).astype('float16')

    #3.
    windows = [7, 14, 28, 30, 45, 60, 90, 120]
    for window in tqdm(windows):
        df_long[f"rmean_28_{window}"] = df_long.groupby(["id"])["lag_28"].transform(lambda x: x.rolling(window).mean()).astype('float16')

    #4.
    df_long = pd.merge(df_long, df_cal.drop(columns=['weekday']), how='left', on='d')
    df_long = pd.merge(df_long, df_price.drop(columns=['store_id', 'item_id']), how='left', on=['id', 'wm_yr_wk'])

    #5.
    df_long['price_shift_t1'] = df_long.groupby(['id'])['sell_price'].shift(1)
    df_long['price_change_t1'] = (df_long['sell_price'] - df_long['price_shift_t1'])/df_long['price_shift_t1']

    
    '''
    Data Manipulation:
    1. Retain data after day 1913 and drop the rest
    2. Convert dtype of lag features to 'int16' 
    3. Replace null values in price based features with 0 
    '''
    
    #1.
    df_long = (df_long[df_long['d']>train_end]).reset_index(drop=True)
    
    #2.
    for feat in ["lag_28", "lag_30", "lag_31", "lag_35", "lag_42", "lag_49", "lag_56", "lag_63", "lag_70", "lag_77"]:
        df_long[feat] = df_long[feat].astype('int16')
    #3.    
    for feat in ['sell_price','price_shift_t1','price_change_t1']:
        df_long[feat] = df_long[feat].replace(np.nan, 0)
    
    '''
    Downcast Final Dataframe
    '''
    df_long = downcast(df_long)
    
    return df_long

# 2. Create Final Pipelines

## 2.1 Forecast Pipeline

In [60]:
#sales forecasting for evaluation days i.e. 1942 to 1969
def forecastPipe(X):
    ''' Takes list of level 12 ids as input and returns sales forecast for days 1942 to 1969 '''
    train_start, train_end = 1069, 1941
    
    #load the model
    model = pickle.load(open("model_eval.pkl", "rb"))
    
    #get the long format dataframe with added features to be used as input to the model
    print("<<<Preprocessing>>>")
    X_q = featEng_eval(X)
    
    #create a list of features to be selected
    removeFeat = ['d', 'sales', 'date', 'wm_yr_wk']
    features = [c for c in X_q.columns if c not in removeFeat]
    
    df_sales = pd.read_csv('sales_train_evaluation.csv')
    
    #create a forecast dataframe with id column containing the ids in X
    df_q = df_sales.loc[df_sales['id'].isin(X), ['id']]
    
    #generate forecasts
    print("<<<Generating Forecasts>>>")
    for day in range(1,29):       
        df_q[f"F{day}"] = model.predict(X_q.loc[X_q["d"]==(train_end+day), features])
        
    df_q.reset_index(drop=True, inplace=True)
    print("<<<Forecast Dataframe Created>>>")
    
    return df_q

## 2.2 Performance Metric Pipeline

In [49]:
#sales forecasting and computing performance for validation days i.e. 1914 to 1941
def performancePipe(X, y):
    ''' Takes level 12 series sales data along with target as input and returns WRMSSE score 
        on forecast made for days 1914 to 1941 '''
    
    train_start, train_end = 1069, 1913
    #load the model
    model = pickle.load(open("model_val.pkl", "rb"))

    #get the long format dataframe with added features to be used as input to the model
    print("<<<Preprocessing>>>")
    X_val = featEng_val(X)
    
    #create a list of features to be selected
    removeFeat = ['d', 'sales', 'date', 'wm_yr_wk']
    features = [c for c in X_val.columns if c not in removeFeat]
    
    #load calendar and price dataframe and perform basic data manipulation
    df_cal = pd.read_csv('calendar.csv', parse_dates=['date'])
    df_cal["d_"]=df_cal["d"].apply(lambda x: int(x.split("_")[1]))
    
    df_price = pd.read_csv('sell_prices.csv')
    df_price["id"] = df_price["item_id"] + "_" + df_price["store_id"] + "_evaluation"
    
    #concat historical sales data and target data for wrmsse calculation
    concat_df = pd.concat([X,y], axis=1)
    
    #add forecast data to the same concat dataframe
    print('<<<Generating Forecasts>>>')
    for day in range(1,29):
        concat_df[f"F{day}"] = model.predict(X_val.loc[X_val["d"]==(train_end+day), features])
    
    print('<<<Calculating Weights>>>')
    concat_df = calculate_weightsL12(concat_df, df_cal, df_price, (train_end - (28 - 1), train_end+1)) 
    agg_df = calculate_weightsALL(concat_df, level_groupings)

    print('<<<Calculating WRMSSE Score>>>')
    train_days, val_days, forecast_days = get_day_splits(concat_df, train_start, train_end, val_start = train_end + 1, val_end = train_end + 28)
    wrmsse = WRMSSE(concat_df, agg_df, train_days, val_days, forecast_days,  n = ((train_end - train_start)+1) , h = 28)
    print('<<<Model Evaluated>>>')
    
    return wrmsse

# 3. Check the output

In [35]:
df_sales = pd.read_csv('sales_train_evaluation.csv')
df_price = pd.read_csv('sell_prices.csv')
df_cal = pd.read_csv('calendar.csv', parse_dates=['date'])

## 3.1 Forecast Pipeline

In [54]:
X = ['HOBBIES_1_004_CA_1_evaluation', 'HOUSEHOLD_1_004_CA_1_evaluation', 'FOODS_3_825_WI_3_evaluation']

In [61]:
forecastPipe(X)

<<<Preprocessing>>>


100%|███████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 379.55it/s]


<<<Generating Forecasts>>>
<<<Forecast Dataframe Created>>>


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_004_CA_1_evaluation,1.904641,1.532269,1.585795,1.648829,1.926328,2.320598,2.625637,2.119428,1.699053,...,1.790031,2.283375,2.168417,1.927081,1.420443,1.450095,1.472359,1.613477,2.247674,2.446868
1,HOUSEHOLD_1_004_CA_1_evaluation,1.342333,1.266069,1.035864,1.024562,1.553377,2.073160,1.839534,1.471478,1.334622,...,1.848246,1.741816,1.964079,1.310458,1.431128,1.185328,1.219093,1.659835,2.439339,2.160877
2,FOODS_3_825_WI_3_evaluation,0.716163,0.626379,0.644473,0.647437,0.776059,0.854815,0.882837,0.754092,0.658399,...,0.869791,1.020404,1.035002,0.913090,0.836115,0.908147,0.707877,0.831467,0.881552,0.920081


In [63]:
#checking pipeline results against final submission dataframe (the values are matching)
sub = pd.read_csv('final_submission.csv')
sub[sub['id'].isin(X)]

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30493,HOBBIES_1_004_CA_1_evaluation,1.904641,1.532269,1.585795,1.648829,1.926328,2.320598,2.625637,2.119428,1.699053,...,1.790031,2.283375,2.168417,1.927081,1.420443,1.450095,1.472359,1.613477,2.247674,2.446868
31058,HOUSEHOLD_1_004_CA_1_evaluation,1.342333,1.266069,1.035864,1.024562,1.553377,2.073160,1.839534,1.471478,1.334622,...,1.848246,1.741816,1.964079,1.310458,1.431128,1.185328,1.219093,1.659835,2.439339,2.160877
60977,FOODS_3_825_WI_3_evaluation,0.716163,0.626379,0.644473,0.647437,0.776059,0.854815,0.882837,0.754092,0.658399,...,0.869791,1.020404,1.035002,0.913090,0.836115,0.908147,0.707877,0.831467,0.881552,0.920081


## 3.2 Performance Metric Pipeline

In [36]:
X = df_sales.iloc[:,:-28]
y = df_sales.iloc[:,-28:]

In [50]:
wrmsse_val = performancePipe(X,y)

<<<Preprocessing>>>


100%|████████████████████████████████████████████████████████████████████████████| 8/8 [01:30<00:00, 11.33s/it]


<<<Generating Forecasts>>>
<<<Calculating Weights>>>
<<<Calculating WRMSSE Score>>>
<<<Model Evaluated>>>


In [51]:
print('The Local WRMSSE on Validation:', wrmsse_val)

The Local WRMSSE on Validation: 0.5717035288102513
